## ***מחברת קוד לעיבוד נתוני תוצאות בקטוכם - ניטור מי תהום***

יש לרשום את שמות התחנות **משמאל לימין**

ניתן לשנות את שם הודגם, שיטת הדיגום, ערך סף הגילוי ואת שם הקובץ בעת הצורך.

יש לבחור קבצים ולהריץ את הקוד - ניתן לבחור **כמה קבצי תוצאות** של אותה תחנה בפעם אחת

**לאחר הרצת הקוד יש לוודא כי המרת השמות מהקובץ לשמות שבחרת בקוד נעשה כמו שצריך.** ההמרה תופיע מתחת ללחצן בו העלת את הקבצים. ההמרה תופיע בצורה הבאה לדוגמא:

    "דיגום מי תהום-סונול מעיליא -נקודה 1 P-1": "P-1",

    "דיגום מי תהום-סונול מעיליא -נקודה 4 P-2": "P-2",

    "דיגום מי תהום-סונול מעיליא -נקודה 2 T-1": "T-1",

    "דיגום מי תהום-סונול מעיליא -נקודה 3 T-2": "T-2"




In [1]:
#לרשום את שמות התחנות משמאל לימין, כל שם בגרשיים ופסיק מפריד
drills_list = ['P-1','P-2','T-1','T-2']

In [15]:
#@title נא לבחור פרמטרים מתאימים  { run: "auto" }

dogem_name = 'מר עבדאללה חמדאן' # @param {type:"string"}
digum_method = 'Low flow' # @param {type:"string"}
Not_Detected = "<0.001" # @param {type:"string"}

Font_size = 9 # @param {type:"slider", min:6, max:24, step:1}
Font_name = 'David' # @param {type:"string"}

# Parameters for English text formatting
Font_size_english = 8 # @param {type:"slider", min:6, max:24, step:1}
Font_name_english = 'Times New Roman' # @param {type:"string"}

file_name = 'Example_Bactochem' # @param {type:"string"}

!pip install openpyxl
import pandas as pd
from openpyxl.styles import Font, Alignment
import re
import json

#רק להריץ

In [13]:
#@title להעלות קבצי תוצאות של בקטוכם בלחיצה על הכפתור למטה
from google.colab import files
uploaded = files.upload()

filenames = list(uploaded.keys())

# Initialize an empty list to store the dataframes
dataframes = []

# Loop through each file and read it based on its extension
for filename in filenames:
    if filename.endswith('.csv'):
        df = pd.read_csv(filename, usecols=list(range(21)))
    elif filename.endswith('.xlsx'):
        df = pd.read_excel(filename, usecols=list(range(21)))
    else:
        print(f"Unsupported file format: {filename}")
        continue
    dataframes.append(df)

# Concatenate all dataframes into one
if dataframes:
    df_res_all = pd.concat(dataframes, ignore_index=True)
else:
    print("No valid files uploaded.")

df_res_all = df_res_all[['רכיב', 'תוצאה','תיאור דוגמה','הערות לדוגמא','תאריך דיגום']]

df_res_all = df_res_all[~df_res_all['תיאור דוגמה'].str.contains('בלנק', na=False)]

# Extract unique strings from the column 'תיאור דוגמה'
drills_list_unique = df_res_all['תיאור דוגמה'].dropna().unique().tolist()

# Initialize an empty dictionary
drills_dic = {}

# Iterate through drills_list and check if each name is in drills_list_unique
for drill in drills_list:
  for drill_uni in drills_list_unique:
    if drill in drill_uni:
        # Add to the dictionary with a value (you can customize this value)
        drills_dic[drill_uni] = drill

# Check if the dictionary length is not equal to the length of drills_list
if len(drills_dic) != len(drills_list):
    for drill in drills_list:
        for drill_uni in drills_list_unique:
            # Additional condition: check if drill_uni contains drill[0] and drill[-1]
            if drill[0] in drill_uni and drill[-1] in drill_uni:
                # Add to the dictionary if not already added
                drills_dic.setdefault(drill_uni, drill)

# Print the resulting dictionary
print(json.dumps(drills_dic, indent=4, ensure_ascii=False))


# Replace the values in 'רכיב' column using the mapping dictionary
df_res_all['תיאור דוגמה'] = df_res_all['תיאור דוגמה'].replace(drills_dic)

translation_dict  = {
    'template': [ 'pH', 'EC', 'טמפרטורה', 'חמצן מומס', 'עכירות','רדוקס','עומק פני המים',
              'עומק כללי של הקידוח','עומק דגימה מפני המים'],
    'res_all': ['pH (after stabilization)', 'Elctr.conductivity (after stabilization)',
              'Temp (after stabilization)', 'Dissolved O2 (after stabilization)',
               'Turbidity (after stabilization)','Redox (after stabilization)','Depth of upper level',
              'Total depth of drilling','Sampling depth']
}

translation_dict_bayler  = {
    'template': [ 'עומק פני המים', 'עומק כללי של הקידוח'],
    'res_all': ['depth of upper level', 'total depth of drilling']
}

# Create a mapping dictionary from translation_dict
mapping_dict = dict(zip(translation_dict['res_all'], translation_dict['template']))

# Create a mapping dictionary from translation_dict_bayler
mapping_dict_bayler = dict(zip(translation_dict_bayler['res_all'], translation_dict_bayler['template']))

# Replace the values in 'רכיב' column using the mapping dictionary
df_res_all['רכיב'] = df_res_all['רכיב'].replace(mapping_dict)
df_res_all['רכיב'] = df_res_all['רכיב'].replace(mapping_dict_bayler)

translation_dict_TPH  = {
    'template': [ 'בנזן', 'טולואן', 'אתיל בנזן', 'קסילן ', 'MTBE','עומק פני המים'],
    'res_all': ['Benzene', 'Toluene', 'Ethyl Benzene', 'Xylene', 'MTBE','עומק פני המים']
}

# Create a mapping dictionary from translation_dict
mapping_dict = dict(zip(translation_dict_TPH['res_all'], translation_dict_TPH['template']))

# Replace the values in 'רכיב' column using the mapping dictionary
df_res_all['רכיב'] = df_res_all['רכיב'].replace(mapping_dict)


data = {
    'בדיקה': ['שיטת הדיגום','שם הדוגם', 'זמן שאיבה','נפח שאיבה',
              'pH', 'EC', 'טמפרטורה', 'חמצן מומס', 'עכירות','רדוקס','עומק פני המים',
              'עומק כללי של הקידוח','עומק דגימה מפני המים','צבע המים','ריח המים','תאריך'],
    'יחידות': ['-','-','Minutes','Liter','-',
               'μS/cm', '˚C', 'mg/l', 'NTU','mV','m','m','m','-','-','-']
}

# Create the DataFrame
df_template = pd.DataFrame(data)

for col in drills_list:
    df_template[col] = None


# Fill the empty rows in df_template
for col in drills_list:
    col_res = col
    for test in translation_dict['template']:

        # Find the corresponding value in df_res_all
        value = df_res_all['תוצאה'][(df_res_all['תיאור דוגמה'].str.contains(col_res, na=False)) & (df_res_all['רכיב'] == test)]

        # Check if value is not empty
        if not value.empty:
            # Safely extract the first value
            value_only = value.values[0]

            # Assign the value to the appropriate cell in df_template
            df_template.loc[df_template['בדיקה'] == test, col] = value_only

    # Get values for 'עומק דגימה מפני המים' and 'עומק פני המים'
    water_surface_value = float(df_template.loc[df_template['בדיקה'] == 'עומק פני המים', col].values)

    # Check if 'עומק דגימה מפני המים' value exists and is not NaN
    sample_depth_values = df_template.loc[df_template['בדיקה'] == 'עומק דגימה מפני המים', col].values
    if sample_depth_values.size > 0 and pd.notna(sample_depth_values[0]):
        sample_depth_value = float(sample_depth_values[0])
        # Calculate the difference and update the DataFrame
        df_template.loc[df_template['בדיקה'] == 'עומק דגימה מפני המים', col] = sample_depth_value - water_surface_value
    else:
        sample_depth_value = None  # Handle missing sample depth value appropriately
        df_template.loc[df_template['בדיקה'] == 'עומק דגימה מפני המים', col] = pd.NA


    # Find the corresponding value in df_res_all
    value = df_res_all['תאריך דיגום'][(df_res_all['תיאור דוגמה'].str.contains(col_res, na=False))]
    value_only = value.values[0]
    df_template.loc[df_template['בדיקה'] == 'תאריך', col] = value_only

# Set the string 'שקוף' for all columns in the row 'צבע המים'
df_template.loc[df_template['בדיקה'] == 'צבע המים', drills_list] = 'שקוף'

# Set the string 'ללא ריח' for all columns in the row 'ריח המים'
df_template.loc[df_template['בדיקה'] == 'ריח המים', drills_list] = 'ללא ריח'

# Set the string 'Low flow' for all columns in the row 'שיטת הדיגום'
df_template.loc[df_template['בדיקה'] == 'שיטת הדיגום', drills_list] = digum_method

# Set the string 'Low flow' for all columns in the row 'שיטת הדיגום'
df_template.loc[df_template['בדיקה'] == 'שם הדוגם', drills_list] = dogem_name

# Extract the 'תאריך' value
date_value = df_template.loc[df_template['בדיקה'] == 'תאריך', df_template.columns != 'בדיקה'].squeeze()

# Convert date_value to the required date format if necessary
date_value = pd.to_datetime(date_value, errors='coerce').dt.strftime('%d.%m.%y')

# Create df_table_2 from df_template
df_table_2 = df_template[df_template['בדיקה'].isin(translation_dict['template'])]

# Transpose the DataFrame
df_table_2 = df_table_2.set_index('בדיקה').transpose()

# Add the 'תאריך' column to df_table_2
df_table_2.insert(0, 'תאריך', date_value)

# Reset the index
df_table_2 = df_table_2.reset_index()

# Rename the 'index' column to 'קידוח'
df_table_2 = df_table_2.rename(columns={'index': 'קידוח'})

df_table_2['תאריך'] = pd.to_datetime(df_table_2['תאריך']).dt.strftime('%d.%m.%y')

df_table_2['תאריך'][0] = '-'



dataֹֹֹֹ_TPH = {
    'מרכיב': [ 'בנזן', 'טולואן', 'אתיל בנזן', 'קסילן ', 'MTBE','עומק פני המים' ],
    'יחידות': ['mg/L','mg/L','mg/L','mg/L','mg/L', 'm']
}

# Create the DataFrame
df_template_TPH = pd.DataFrame(dataֹֹֹֹ_TPH)

for col in drills_list:
    df_template_TPH[col] = None

# Fill the empty rows in df_template
for col in drills_list:
    col_res = col
    for test in translation_dict_TPH['template']:

        # Find the corresponding value in df_res_all
        value = df_res_all['תוצאה'][(df_res_all['תיאור דוגמה'].str.contains(col_res, na=False)) & (df_res_all['רכיב'] == test)]
        value_only = value.values[0]

        df_template_TPH.loc[df_template_TPH['מרכיב'] == test, col] = value_only

# Create df_table_3 from df_template
df_table_3 = df_template_TPH[df_template_TPH['מרכיב'].isin(translation_dict_TPH['template'])]

# Transpose the DataFrame
df_table_3 = df_table_3.set_index('מרכיב').transpose()

# Add the 'תאריך' column to df_table_3
df_table_3.insert(0, 'תאריך', date_value)

# Add the 'תאריך' column to df_table_3
df_table_3.insert(1, 'מרכיב', None)

# Reset the index
df_table_3 = df_table_3.reset_index()

# Rename the 'index' column to 'קידוח'
df_table_3 = df_table_3.rename(columns={'index': 'קידוח'})

df_table_3['תאריך'] = pd.to_datetime(df_table_3['תאריך']).dt.strftime('%d.%m.%y')

df_table_3['תאריך'][0] = '-'

df_table_3 = df_table_3.replace('Not Detected', Not_Detected)

df_template = df_template[~df_template.isin(['תאריך']).any(axis=1)]

# Define the row names for the new table (Sheet 4) in the desired order
row_names_sheet_4 = ['טמפרטורה', 'עכירות', 'חמצן מומס', 'pH', 'רדוקס', 'EC', 'MTBE', 'TBA', 'בנזן', 'אתיל בנזן', 'טולואן', 'קסילן ']

# Create a new DataFrame for Sheet 4
df_template_sheet_4 = pd.DataFrame({
    'בדיקה': row_names_sheet_4,  # Set the row names in Hebrew and English
    'יחידות': ['˚C', 'NTU', 'mg/l', '-', 'mV', 'mS/cm', 'μg/L', '-', 'μg/L', 'μg/L', 'μg/L', 'μg/L']  # Corresponding units
})

# Add empty columns for each drill in drills_list
for col in drills_list:
    df_template_sheet_4[col] = None

# Predefined Not_Detected value
#Not_Detected = "<0.001"  # Example

# Helper function to check if a value is numeric
def is_numeric(value):
    try:
        float(value)  # Try converting to float
        return True
    except (ValueError, TypeError):
        return False

# Helper function to calculate the adjusted Not_Detected value
def adjust_not_detected(not_detected_value, multiplier=1000):
    try:
        # Extract the numeric portion and multiply it
        numeric_part = float(not_detected_value.strip("<")) * multiplier
        return f"<{numeric_part}"
    except (ValueError, TypeError):
        return not_detected_value  # Return the original if conversion fails

# Fill the empty rows in df_template_sheet_4
for col in drills_list:
    col_res = col

    # Loop through row names in Sheet 4 and fill values
    for test in row_names_sheet_4:

        # Special handling for MTBE, Benzene (בנזן), Ethyl Benzene (אתיל בנזן), Toluene (טולואן), Xylene (קסילן) -> from df_table_3
        if test in ['MTBE', 'בנזן', 'אתיל בנזן', 'טולואן', 'קסילן ']:
            value = df_table_3[test][df_table_3['קידוח'].str.contains(col_res, na=False)]
            if not value.empty:
                if is_numeric(value.values[0]):
                    value_only = float(value.values[0]) * 1000  # Multiply these specific values by 1000
                else:
                    value_only = adjust_not_detected(Not_Detected)  # Adjust the Not_Detected value
                df_template_sheet_4.loc[df_template_sheet_4['בדיקה'] == test, col] = value_only

        elif test == 'EC':  # Special handling for EC
            value = df_res_all['תוצאה'][(df_res_all['תיאור דוגמה'].str.contains(col_res, na=False)) & (df_res_all['רכיב'] == test)]
            if not value.empty:
                if is_numeric(value.values[0]):
                    value_only = float(value.values[0]) / 1000  # Divide EC values by 1000
                else:
                    value_only = adjust_not_detected(Not_Detected)  # Adjust the Not_Detected value
                df_template_sheet_4.loc[df_template_sheet_4['בדיקה'] == test, col] = value_only

        else:
            # For other tests, find the corresponding value in df_res_all
            value = df_res_all['תוצאה'][(df_res_all['תיאור דוגמה'].str.contains(col_res, na=False)) & (df_res_all['רכיב'] == test)]
            if not value.empty:
                if is_numeric(value.values[0]):
                    value_only = float(value.values[0])  # Convert value to float for consistency
                else:
                    value_only = adjust_not_detected(Not_Detected)  # Adjust the Not_Detected value
                df_template_sheet_4.loc[df_template_sheet_4['בדיקה'] == test, col] = value_only

from openpyxl import load_workbook
from google.colab import files

# Create a Pandas Excel writer using openpyxl as the engine
with pd.ExcelWriter(f'{file_name}.xlsx', engine='openpyxl') as writer:
    # Write each DataFrame to a different sheet
    df_template.to_excel(writer, sheet_name='Table_1', index=False)
    df_table_2.to_excel(writer, sheet_name='Table_2', index=False)
    df_table_3.to_excel(writer, sheet_name='Table_3', index=False)
    df_template_sheet_4.to_excel(writer, sheet_name='Table_4', index=False)

# Load the workbook to modify it
workbook = load_workbook(f'{file_name}.xlsx')

# Set the sheet direction to RTL and font style to David, size 9 for each sheet
for sheet in workbook.sheetnames:
    worksheet = workbook[sheet]
    worksheet.sheet_view.rightToLeft = True

    # Set the font for each cell in the sheet
    for row in worksheet.iter_rows():
        for cell in row:
            cell.font = Font(name=Font_name, size=Font_size)

# Regular expression to identify English characters and specific units
english_regex = re.compile(r'^[A-Za-z\s.,]*$')
units_to_check = ['mg/L', 'μS/cm', '˚C', 'NTU', 'mV', 'mg/l']  # Add other units as needed


# Set the sheet direction to RTL, font style, and alignment for each sheet
for sheet in workbook.sheetnames:
    worksheet = workbook[sheet]
    worksheet.sheet_view.rightToLeft = True

    # Set the font and alignment for each cell in the sheet
    for row in worksheet.iter_rows():
        for cell in row:
            # Check if the cell contains only English text or specific units
            if any(unit in str(cell.value) for unit in units_to_check) or english_regex.match(str(cell.value)):
                # Set English font style for specific units or English text
                cell.font = Font(name=Font_name_english, size=Font_size_english)
            elif isinstance(cell.value, (int, float)):
                # Set default Hebrew font style for numbers
                cell.font = Font(name=Font_name, size=Font_size)
            else:
                # Set default Hebrew font style for non-English text (not numbers)
                cell.font = Font(name=Font_name, size=Font_size)

            # Align all cells to the middle
            cell.alignment = Alignment(horizontal='center', vertical='center')

# Save the modified workbook
workbook.save(f'{file_name}.xlsx')

# Download the Excel file
files.download(f'{file_name}.xlsx')

Saving D130524-0013.csv to D130524-0013 (5).csv
{
    "דיגום מי תהום-סונול מעיליא -נקודה 1 P-1": "P-1",
    "דיגום מי תהום-סונול מעיליא -נקודה 4 P-2": "P-2",
    "דיגום מי תהום-סונול מעיליא -נקודה 2 T-1": "T-1",
    "דיגום מי תהום-סונול מעיליא -נקודה 3 T-2": "T-2"
}


<ipython-input-13-e91e9fc312df>:128: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  water_surface_value = float(df_template.loc[df_template['בדיקה'] == 'עומק פני המים', col].values)
<ipython-input-13-e91e9fc312df>:128: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  water_surface_value = float(df_template.loc[df_template['בדיקה'] == 'עומק פני המים', col].values)
<ipython-input-13-e91e9fc312df>:128: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  water_surface_value = fl

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>